# Choc / détente sur paroi

On vous propose de mettre en place ...

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [4]:
import math
import numpy as np
import matplotlib.pyplot as plt
from hades.common import defaultgas
from hades.aero import degree     as deg  # import trigo functions with degree unit support
from hades.aero import ShockWave  as sw   # import functions for shockwave computation
from hades.aero import Supersonic as sup  # import functions for steady supersonic flows
%matplotlib inline

On définit tout d'abord les paramètres de ce cas. Ils sont définis comme des variables globales, que l'on peut utiliser directement dans les fonctions (comme $\gamma$ par exemple).

In [49]:
# definition of problem parameters
gam  = 1.4 ; defaultgas.set_gamma(gam)
M0   = 2.5
wang1=  26.
wang2= -30.

# function to plot the geometry
#
def plot_geom(xneg=-.5, length=1., zoom=1, ymax=1.5):
    fig = plt.figure(figsize=(14*zoom,8*zoom))
    ax  = fig.add_subplot(111) 
    #plt.axis([xneg, length])
    ax.set(aspect="equal", xlim=[xneg, 4*length], ylim=[-.1, ymax])
    h = length*deg.tan(wang1)
    #plt.plot([xneg, length],    [1, 1],                      color="black", linewidth=2)
    plt.plot([xneg, 0, length, length-h/deg.tan(wang2), 4*length], 
             [0, 0, h, 0, 0], color="black", linewidth=2)
#
# test de la fonction de tracé   
plot_geom(zoom=.5)

## Premier choc attaché

In [50]:
devmax   = sw.dev_Max(M0)
devsonic = sw.dev_Sonic(M0)
print(("For upstream Mach number M0= {:1.4},\n* maximum deviation is {:1.4}°\n"+
      "* limit for downstream supersonic flow is {:1.4}°").format(M0, devmax, devsonic))

In [51]:
wdev = wang1
sig1 = sw.sigma_Mach_deflection(M0, wdev)
Mn0  = M0*deg.sin(sig1)
p1p0 = sw.Ps_ratio(Mn0)
Mn1  = sw.downstream_Mn(Mn0)
M1   = Mn1/deg.sin(sig1-wdev)
print("shock with {:1.4}° deviation and angle {:1.4}\ndownstream Mach number is M1= {:1.4}\nCompression ratio   is p1/p0= {:1.4}".format(wdev, sig1, M1, p1p0))

In [52]:
plot_geom()
# tracé du choc
x=1.5
plt.plot([0, x], [0, x*deg.tan(sig1)], linewidth=2, color='red')
# tracé des caractéristiques arrivant en x=.8
x = .8
y = x*deg.tan(sig1)
plt.plot([x, x-y/deg.tan(deg.asin(1./M0))], [y, 0], 'blue')
plt.plot([x, x-y/deg.tan(wang1+deg.asin(1./M1))], [y, 0], 'blue')

## Détente



In [53]:
wdev = wang2-wang1
om1 = sup.PrandtlMeyer_Mach(M1)
om2 = om1 - wdev
M2  = sup.Mach_PrandtlMeyer(om2)
p2p0 = p1p0*sup.IsentropicPsratio_Mach_deflection(M1, wdev)
#
plot_geom()
# tracé du choc
x=1.5
plt.plot([0, x], [0, x*deg.tan(sig1)], linewidth=2, color='red')
# tracé des caractéristiques de la détente
x = 1
y = x*deg.tan(wang1)
plt.plot([x, x+2], [y, y+2*deg.tan(wang1+deg.asin(1./M1))], 'blue')
plt.plot([x, x+2], [y, y+2*deg.tan(wang2+deg.asin(1./M2))], 'blue')


## Dernier choc



In [54]:
wang3 = 0.
wdev=wang3-wang2
#
devmax   = sw.dev_Max(M2)
devsonic = sw.dev_Sonic(M2)
print(("For upstream Mach number M2= {:1.4},\n* maximum deviation is {:1.4}°\n"+
      "* limit for downstream supersonic flow is {:1.4}°").format(M2, devmax, devsonic))
#
sig2 = sw.sigma_Mach_deflection(M2, wdev)
Mn2  = M2*deg.sin(sig2)
Mn3  = sw.downstream_Mn(Mn2)
M3   = Mn3/deg.sin(sig2-wdev)
p3p0 = p2p0*sw.Ps_ratio(Mn2)
print("shock with {:1.4}° deviation and angle {:1.4}\ndownstream Mach number is M3= {:1.4}".format(wdev, sig2, M3))

In [55]:
plot_geom(ymax=2.)
# tracé du choc
x=2
plt.plot([0, x], [0, x*deg.tan(sig1)], linewidth=2, color='red')
# tracé des caractéristiques de la détente
x = 1
y = x*deg.tan(wang1)
plt.plot([x, x+2], [y, y+2*deg.tan(wang1+deg.asin(1./M1))], 'blue')
plt.plot([x, x+3], [y, y+3*deg.tan(wang2+deg.asin(1./M2))], 'blue')
#
# tracé du choc
x=1-deg.tan(wang1)/deg.tan(wang2)
plt.plot([x, x+2], [0, 2*deg.tan(wang2+sig2)], linewidth=2, color='red')


# Représentation dans la polaire

In [56]:
import hades.aero.plot.shockpolar as swplt

fig=swplt.figure_theta_pressure(figsize=(14,8))
fig.suptitle('Polar of Shock-Waves, $\gamma = %.1f$'%gam, fontsize=12, y=0.93)
plt.xlabel('flow angle', fontsize=10)
plt.ylabel('normalized static pressure', fontsize=10)
if p2p0 < 20.: plt.yscale('linear') # default is logarithmic
#
# plot polar curves
swplt.plot_theta_pressure(M0, devmax=True, sonic=True)
swplt.plot_theta_pressure(M2, thet_init=wang2, p_init=p2p0, color='red')

# plot symbols for flow regions
plt.plot(0,     1.,   'bo')
plt.plot(wang1, p1p0, 'yo')
plt.plot(wang2, p2p0, 'yo')
plt.plot(wang3, p3p0, 'ro')

---

<a id="ipython"></a>
## Ipython et notebook : usage

* le notebook utilise la langage de base python en version améliorée, Ipython, qui permet la complétion des noms (variables, fonctions, modules) avec la touche tabulation
* toutes les cellules peuvent être modifiées par un double-clic et sont réinterprêtées avec `shift-entrée`
* l'ensemble de la feuille peut être exécutée avec le menu `Cell/run all cells`
* **n'oubliez pas de sauvegarder régulièrement votre feuille** (bouton _enregistrer_)


In [8]:
from IPython.core.display import HTML ; HTML(open("./custom.css", "r").read()) # notebook style